In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model 
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate, Conv2D, MaxPool2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyperclip

In [138]:
# 불러오기
result = pd.read_csv("data/떡볶이2000-1.csv")

In [139]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                2000 non-null   object 
 1   Contents          2000 non-null   object 
 2   Tags              1817 non-null   object 
 3   Like              2000 non-null   object 
 4   Like_avg          2000 non-null   int64  
 5   Like_mid          2000 non-null   int64  
 6   tags_count        2000 non-null   int64  
 7   tag_sum           2000 non-null   int64  
 8   tag_rate          2000 non-null   float64
 9   Like_avg_normal1  2000 non-null   float64
 10  Like_avg_normal2  2000 non-null   float64
 11  Like_mid_normal1  2000 non-null   float64
 12  Like_mid_normal2  2000 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 203.2+ KB


In [140]:
result["Tags"] = result["Tags"].fillna("")

In [141]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                2000 non-null   object 
 1   Contents          2000 non-null   object 
 2   Tags              2000 non-null   object 
 3   Like              2000 non-null   object 
 4   Like_avg          2000 non-null   int64  
 5   Like_mid          2000 non-null   int64  
 6   tags_count        2000 non-null   int64  
 7   tag_sum           2000 non-null   int64  
 8   tag_rate          2000 non-null   float64
 9   Like_avg_normal1  2000 non-null   float64
 10  Like_avg_normal2  2000 non-null   float64
 11  Like_mid_normal1  2000 non-null   float64
 12  Like_mid_normal2  2000 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 203.2+ KB


In [122]:
result.iloc[798,1] = 0
result.iloc[798,1]

0

In [142]:
result= result.drop(['Like', 'Like_avg', 'Like_avg_normal1', 'Like_avg_normal2', 'Like_mid_normal1'], axis=1)
result

,ID,Contents,Tags,Like_mid,tags_count,tag_sum,tag_rate,Like_mid_normal2
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,14,23,60.87,0.073878
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,6,11,54.55,0.123679
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,0,5,0.00,0.211534
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,2,7,28.57,0.164787
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,15,11,136.36,0.309255
...,...,...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",15,11,19,57.89,0.001762
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,17,1,1,100.00,0.001997
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",22,6,30,20.00,0.002584
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",8,8,15,53.33,0.000940


In [143]:
result = result.rename(columns={'tag_sum' : 'Tags_sum', 'tags_count' : 'Tags_count', 'tag_rate' : 'Tags_rate', 'Like_mid' : 'Likes', 
                               'Like_mid_normal2' : 'Scaled_likes'})
result

,ID,Contents,Tags,Likes,Tags_count,Tags_sum,Tags_rate,Scaled_likes
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,14,23,60.87,0.073878
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,6,11,54.55,0.123679
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,0,5,0.00,0.211534
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,2,7,28.57,0.164787
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,15,11,136.36,0.309255
...,...,...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",15,11,19,57.89,0.001762
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,17,1,1,100.00,0.001997
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",22,6,30,20.00,0.002584
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",8,8,15,53.33,0.000940


In [144]:
result["Category"] = "떡볶이"
result

,ID,Contents,Tags,Likes,Tags_count,Tags_sum,Tags_rate,Scaled_likes,Category
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,14,23,60.87,0.073878,떡볶이
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,6,11,54.55,0.123679,떡볶이
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,0,5,0.00,0.211534,떡볶이
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,2,7,28.57,0.164787,떡볶이
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,15,11,136.36,0.309255,떡볶이
...,...,...,...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",15,11,19,57.89,0.001762,떡볶이
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,17,1,1,100.00,0.001997,떡볶이
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",22,6,30,20.00,0.002584,떡볶이
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",8,8,15,53.33,0.000940,떡볶이


In [145]:
result = result.reindex(columns=['Category', 'ID', 'Contents', 'Tags', 'Tags_sum', 'Tags_count', 'Tags_rate', 'Likes', 'Scaled_likes'])
result

,Category,ID,Contents,Tags,Tags_sum,Tags_count,Tags_rate,Likes,Scaled_likes
0,떡볶이,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",23,14,60.87,629,0.073878
1,떡볶이,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",11,6,54.55,1053,0.123679
2,떡볶이,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",5,0,0.00,1801,0.211534
3,떡볶이,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",7,2,28.57,1403,0.164787
4,떡볶이,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",11,15,136.36,2633,0.309255
...,...,...,...,...,...,...,...,...,...
1995,떡볶이,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",19,11,57.89,15,0.001762
1996,떡볶이,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,1,1,100.00,17,0.001997
1997,떡볶이,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",30,6,20.00,22,0.002584
1998,떡볶이,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",15,8,53.33,8,0.000940


In [146]:
# 저장
result.to_csv("data/떡볶이_R01.csv", encoding="utf-8", index=False)

In [151]:
import pandas as pd
import glob
import os

input_file = r'C:\mhpark\01.PythonWork\project2\data' # csv파일들이 있는 디렉토리 위치
output_file = r'C:\mhpark\01.PythonWork\project2\data\result2.csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, '*_R02.csv')) # glob함수로 sales_로 시작하는 파일들을 모은다
print(allFile_list)
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트를 하나 만든다
for file in allFile_list:
    df = pd.read_csv(file) # for구문으로 csv파일들을 읽어 들인다
    allData.append(df) # 빈 리스트에 읽어 들인 내용을 추가한다

dataCombine = pd.concat(allData, axis=0, ignore_index=True) # concat함수를 이용해서 리스트의 내용을 병합
# axis=0은 수직으로 병합함. axis=1은 수평. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 한다.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장한다. 인덱스를 빼려면 False로 설정

['C:\\mhpark\\01.PythonWork\\project2\\data\\돈까스_R02.csv', 'C:\\mhpark\\01.PythonWork\\project2\\data\\떡볶이_R02.csv', 'C:\\mhpark\\01.PythonWork\\project2\\data\\스시_R02.csv', 'C:\\mhpark\\01.PythonWork\\project2\\data\\치킨_R02.csv', 'C:\\mhpark\\01.PythonWork\\project2\\data\\파스타_R02.csv', 'C:\\mhpark\\01.PythonWork\\project2\\data\\피자_R02.csv', 'C:\\mhpark\\01.PythonWork\\project2\\data\\햄버거_R02.csv']


In [152]:
# 불러오기
result = pd.read_csv("data/result2.csv", error_bad_lines=False)

C:\Users\mhpark\AppData\Local\Temp\ipykernel_26056\211986362.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  result = pd.read_csv("data/result2.csv", error_bad_lines=False)


In [1]:
result

NameError: name 'result' is not defined

In [93]:
# 불러오기
result = pd.read_csv("data/떡볶이계정_15.csv")
result

,Post_num,Followers
0,49,21
1,9,41
2,487,341
3,7703,4917
4,77,137
...,...,...
95,51,31
96,4030,312
97,631,70
98,371,1850


In [90]:
result.drop(['Unnamed: 0'], axis = 1, inplace = True)
result

KeyError: "['Unnamed: 0'] not found in axis"

In [87]:
result.columns=["Post_num", "Followers"]
result

,Post_num,Followers
0,1.4만,5363
1,5724,3424
2,131,135
3,177,441
4,21,77
...,...,...
95,868,675
96,0,0
97,4049,28.6만
98,120,94


In [88]:
# 저장
result.to_csv("data/떡볶이계정_15.csv", encoding="utf-8", index=False)